<a href="https://colab.research.google.com/github/alammobaDar/Data_Scraping/blob/main/ACTIVITY5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***INSTALL MODULES***

In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install pandas

# ***IMPORT MODULES***

In [157]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import re
import csv

ModuleNotFoundError: No module named 'file'

# ***EXTRACT DATA***

In [158]:
urls = [
    'https://marvelcinematicuniverse.fandom.com/wiki/Steve_Rogers'
]

def clean_text(text):
  unwanted_patterns = [
    r'In chronological order:',
    r'In an indefinite temporality:',
    r'In narrative order:',
    r'\[\d+\]\n',


    ]
  for pattern in unwanted_patterns:
    text = re.sub(pattern, '', text)
  return text.strip()

def extract_data(url):
  headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
  }
  response = requests.get(url, headers=headers)
  if response.status_code == 200:

    soup = BeautifulSoup(response.text, 'lxml')
    p_tags = soup.find("div", {'class':'mw-parser-output'}).find_all('p')
    info = []
    for p in p_tags:
      if p.find('aside', {'class':'portable-infobox pi-background pi-border-color pi-theme-wikia pi-layout-default'}):
        continue
      if p.find_parent('figure'):
        continue
      if p.find('sup', {'class':'reference'}):
        continue
      text = p.get_text(strip = False)
      cleaned_text = clean_text(text)
      if cleaned_text == '':
        continue
      info.append(text)
  return info

data = [extract_data(url) for url in urls]
flat_data = [item for sublist in data if sublist for item in sublist]


df_info = pd.DataFrame(flat_data, columns=['Info'])
# Create a DataFrame with each item as a row
df_info.index = [f"{i+1}." for i in range(len(flat_data))]

# Save DataFrame to TXT file with custom separator and without quotation marks
df_info.to_csv(
    'captain_america_dataset_Info.txt',
    sep=' ',                # Use a single space as the column separator
    index=True,            # Include the DataFrame index
    header=False,          # Exclude column names
    quoting=csv.QUOTE_NONE,  # Disable quoting of strings
    escapechar='\t'
)


print(df_info)

                                                   Info
1.    Steven Grant "Steve" Rogers is a World War II ...
2.    In 1943, Rogers rejoined the SSR after single-...
3.    When he awoke, Rogers found himself alone in a...
4.    Rogers then joined S.H.I.E.L.D. as a counter-t...
5.    Rogers and the Avengers reassembled and embark...
...                                                 ...
362.  He does have a darker side, however, with Ultr...
363.  Rogers has a strong dislike for agencies or gr...
364.  The Snap had a profound effect on Rogers' opti...
365.  Artificially Enhanced Physiology: The Super So...
366.  As a weapon of choice, Rogers uses a round sha...

[366 rows x 1 columns]


In [ ]:
df_info

In [167]:
all_sentences = []

pattern = r'(?<!\b[A-Z])\. (?=[A-Z])'

for sentences in df_info['Info']:
  sentences_split = re.split(pattern, sentences)

  for sentences_ in sentences.split('. '):
    for sentence in sentences_.split('. '):

      if sentence.split() == []:
        continue
      else:
        all_sentences.append(sentence)
df_info_ = pd.DataFrame(all_sentences, columns=['Info'])
df_info_.index = [f"{i+1}." for i in range(len(df_info_))]
df_info_.to_csv(
    'captain_america_dataset_Info.txt',
    sep=' ',                # Use a single space as the column separator
    index=True,            # Include the DataFrame index
    header=False,          # Exclude column names
    quoting=csv.QUOTE_NONE,  # Disable quoting of strings
    escapechar='\t'
)

In [161]:
df_info_

,Info
0,"Steven Grant ""Steve"" Rogers is a World War II ..."
1,Rogers grew up suffering from numerous health ...
2,Rogers ultimately volunteered for Project Rebi...
3,The serum greatly enhanced Rogers' physical ab...
4,After Erskine's assassination and being doubte...
...,...
1167,The Super Soldier Serum also healed all of his...
1168,It also prevents him from getting intoxicated ...
1169,The Super Soldier Serum permanently transforme...
1170,"As a weapon of choice, Rogers uses a round sha..."


In [90]:
import re

text = "This is a sentence. This is another sentence. S.H.I.E.L.D. is an acronym. And another one."
# Regex to split on period followed by a space, but not when the period is part of an acronym
pattern = r'(?<!\b[A-Z])\. (?=[A-Z])'

sentences = re.split(pattern, text)
print(sentences)

['This is a sentence', 'This is another sentence', 'S.H.I.E.L.D. is an acronym', 'And another one.']
